### Basic trading

- entry: do I use only the data for that stock endogenous, or also include the data from other stocks, exagenous.
- entry and exit :
- exit : take profit, stop loss, time out

In [83]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf  # for data
from pandas_datareader import data as pdr

In [2]:
yf.pdr_override()

In [108]:
stock = input('Enter stock tikcer symbol:')

Enter stock tikcer symbol: SAVA


In [138]:
stock

'SAVA'

In [144]:
start_year = 2018
start_month = 1
start_day = 1

start = dt.datetime(start_year,start_month,start_day)
now = dt.datetime.now()
df = pdr.get_data_yahoo(stock,period = "60d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "5m",)


emasUsed = [26,50]
for ema in emasUsed:
    df['Ema_' + str(ema)] = round(df['Adj Close'].ewm(span = ema, adjust = False).mean(),2)
    df['Middle Band'] =df['Adj Close'].rolling(window=20).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Close'] > df['Upper Band'],'above_ballinger','normal')
    
df = df.iloc[20:,:]


[*********************100%***********************]  1 of 1 completed


In [145]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,Ema_26,Middle Band,Upper Band,Lower Band,status_lower,status_upper,Ema_50
Datetime,,,,,,,,,,,,,
2020-08-04 11:10:00-04:00,3.1550,3.1600,3.1515,3.16,3.16,4456,3.17,3.160630,3.182882,3.138378,normal,normal,3.19
2020-08-04 11:15:00-04:00,3.1604,3.1604,3.1500,3.16,3.16,2933,3.17,3.160630,3.182882,3.138378,normal,normal,3.19
2020-08-04 11:20:00-04:00,3.1600,3.1600,3.1600,3.16,3.16,305,3.17,3.159630,3.180010,3.139250,normal,normal,3.18
2020-08-04 11:25:00-04:00,3.1600,3.1600,3.1401,3.15,3.15,14815,3.17,3.158630,3.178837,3.138423,normal,normal,3.18
2020-08-04 11:30:00-04:00,3.1500,3.1500,3.1500,3.15,3.15,400,3.17,3.159125,3.177912,3.140338,normal,normal,3.18


In [146]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,Ema_26,Middle Band,Upper Band,Lower Band,status_lower,status_upper,Ema_50
Datetime,,,,,,,,,,,,,
2020-10-27 15:35:00-04:00,9.1600,9.2000,9.1600,9.200,9.200,28803,9.07,9.04394,9.147287,8.940593,normal,above_ballinger,9.08
2020-10-27 15:40:00-04:00,9.1800,9.1999,9.1101,9.125,9.125,20028,9.07,9.04869,9.157679,8.939701,normal,normal,9.08
2020-10-27 15:45:00-04:00,9.1188,9.1700,9.1000,9.100,9.100,19369,9.07,9.05219,9.163053,8.941327,normal,normal,9.08
2020-10-27 15:50:00-04:00,9.1100,9.1400,9.0800,9.110,9.110,17628,9.07,9.05819,9.167912,8.948469,normal,normal,9.08
2020-10-27 15:55:00-04:00,9.1100,9.1500,9.0900,9.110,9.110,48017,9.08,9.06369,9.172201,8.955179,normal,normal,9.08


In [7]:
#### 1. STRATEGY CROSSING

In [147]:
pos = 0
num = 0
percentchange = []
start_of_this_month = int((len(df.index[:])/60)*30)

for i in df.index[start_of_this_month:]:
    cmin = df['Ema_26'][i]
    cmax = df['Ema_50'][i]
    status_lower = df['status_lower'][i]
    
    close = df['Adj Close'][i]
    
    #if(status_lower=='below_ballinger'):
   # if (cmin>cmax) and (status_lower=='below_ballinger'):
    if (cmin>cmax):
       # print('red white blue')
        if pos ==0:
            bp =close
            pos=1
            print('Buying now at'+ str(bp))
            print(i)
            
        
    elif(cmin<cmax):
        #print('blue white red')
        if pos ==1:
            pos = 0
            sp = close
            print('Selling now at'+ str(sp))
            print(i)
            pc = (sp/bp-1)*100
            percentchange.append(pc)

            
    if num ==df['Adj Close'].count()-1 and pos==1:
        pos = 0
        sp = close
        print('Selling now at'+ str(sp))
        pc = (sp/bp-1)*100
        percentchange.append(pc)      
    num +=1

Buying now at6.880000114440918
2020-09-16 14:05:00-04:00
Selling now at6.845900058746338
2020-09-17 11:25:00-04:00
Buying now at7.130000114440918
2020-09-17 14:05:00-04:00
Selling now at9.199999809265137
2020-09-21 10:10:00-04:00
Buying now at9.789999961853027
2020-09-21 15:05:00-04:00
Selling now at9.977700233459473
2020-09-23 10:25:00-04:00
Buying now at9.929699897766113
2020-09-24 12:25:00-04:00
Selling now at9.569999694824219
2020-09-24 15:45:00-04:00
Buying now at10.100000381469727
2020-09-25 09:30:00-04:00
Selling now at10.610099792480469
2020-09-28 10:30:00-04:00
Buying now at10.938400268554688
2020-09-28 12:35:00-04:00
Selling now at11.329999923706055
2020-09-30 10:05:00-04:00
Buying now at11.503800392150879
2020-09-30 11:00:00-04:00
Selling now at11.489999771118164
2020-09-30 14:25:00-04:00
Buying now at11.749799728393555
2020-10-01 10:40:00-04:00
Selling now at11.469900131225586
2020-10-01 12:50:00-04:00
Buying now at10.88010025024414
2020-10-05 10:05:00-04:00
Selling now at1

In [11]:
#### 2. BALLINGER 

In [12]:
pos = 0
num = 0
percentchange = []

for i in df.index:
    cmin = df['Ema_26'][i]
    cmax = df['Ema_50'][i]
    status_upper = df['status_upper'][i]
    status_lower = df['status_lower'][i]

    
    close = df['Adj Close'][i]
    
    if(status_lower=='below_ballinger'):
       # print('red white blue')
        if pos ==0:
            bp =close
            pos=1
            #print('Buying now at'+ str(bp))
            #print(i)
            
        
    elif(status_upper=='upper_ballinger'):
        #print('blue white red')
        if pos ==1:
            pos = 0
            sp = close
            #print('Selling now at'+ str(sp))
            #print(i)
            pc = (sp/bp-1)*100
            percentchange.append(pc)
            
    if num ==df['Adj Close'].count()-1 and pos==1:
        pos = 0
        sp = close
        #print('Selling now at'+ str(sp))
        pc = (sp/bp-1)*100
        percentchange.append(pc)      
    num +=1
    
#print(percentchange)


In [13]:
#### 3. STRATEGY BUY CROSSING, SELL BALLINGER

In [65]:
pos = 0
num = 0
percentchange = []

for i in df.index:
    cmin = df['Ema_26'][i]
    cmax = df['Ema_50'][i]
    status_upper = df['status_upper'][i]
    
    close = df['Adj Close'][i]
    
    #if(status_lower=='below_ballinger'):
   # if (cmin>cmax) and (status_lower=='below_ballinger'):
    if (cmin>cmax):
       # print('red white blue')
        if pos ==0:
            bp =close
            pos=1
            print('Buying now at'+ str(bp))
            print(i)
            
        
    elif(status_upper=='upper_ballinger'):
        #print('blue white red')
        if pos ==1:
            pos = 0
            sp = close
            #print('Selling now at'+ str(sp))
            #print(i)
            pc = (sp/bp-1)*100
            percentchange.append(pc)
            
    if num ==df['Adj Close'].count()-1 and pos==1:
        pos = 0
        sp = close
        #print('Selling now at'+ str(sp))
        pc = (sp/bp-1)*100
        percentchange.append(pc)      
    num +=1
    
#print(percentchange)


Buying now at0.6798999905586243
2020-09-11 10:40:00-04:00


In [ ]:
#### 4. STRATEGY BUY BALLINGER, SELL CROSSING

In [74]:
pos = 0
num = 0
percentchange = []


start_of_this_month = int((len(df.index[:])/60)*30)


new_index = df.index[start_of_this_month:]
for i in new_index:
    cmin = df['Ema_26'][i]
    cmax = df['Ema_50'][i]
    status_lower = df['status_lower'][i]
    
    close = df['Adj Close'][i]
    
    if(status_lower=='below_ballinger'):
   # if (cmin>cmax) and (status_lower=='below_ballinger'):
       # print('red white blue')
        if pos ==0:
            bp =close
            pos=1
            print('Buying now at'+ str(bp))
            print(i)
            
        
    elif(cmin<cmax):
        #print('blue white red')
        if pos ==1:
            pos = 0
            sp = close
            #print('Selling now at'+ str(sp))
            #print(i)
            pc = (sp/bp-1)*100
            percentchange.append(pc)
            
    if num ==df['Adj Close'].count()-1 and pos==1:
        pos = 0
        sp = close
        print('Selling now at'+ str(sp))
        pc = (sp/bp-1)*100
        percentchange.append(pc)      
    num +=1
    
    
    
    
#print(percentchange)

Buying now at2.5399999618530273
2020-10-06 11:05:00-04:00
Buying now at3.3965001106262207
2020-10-08 13:25:00-04:00
Buying now at3.434999942779541
2020-10-08 13:35:00-04:00
Buying now at3.430000066757202
2020-10-09 11:40:00-04:00
Buying now at3.4600000381469727
2020-10-09 14:00:00-04:00
Buying now at3.4600000381469727
2020-10-09 14:30:00-04:00
Buying now at3.380000114440918
2020-10-12 09:30:00-04:00
Buying now at3.122299909591675
2020-10-12 13:05:00-04:00
Buying now at3.244999885559082
2020-10-13 11:20:00-04:00
Buying now at3.0743000507354736
2020-10-14 10:05:00-04:00
Buying now at3.0399999618530273
2020-10-14 10:25:00-04:00
Buying now at3.049999952316284
2020-10-14 12:20:00-04:00
Buying now at3.2149999141693115
2020-10-16 09:30:00-04:00
Buying now at3.180000066757202
2020-10-16 13:45:00-04:00
Buying now at3.190000057220459
2020-10-16 15:45:00-04:00
Buying now at3.180000066757202
2020-10-16 15:55:00-04:00
Buying now at3.069999933242798
2020-10-19 09:55:00-04:00
Buying now at3.000099897

In [87]:
## print results

gains = 0
ng = 0
losses = 0
nl = 0
totallR = 1
for i in percentchange:
    if i >0:
        gains +=i
        ng +=1
    else:
        losses +=i
        nl +=1
    totallR = totallR*((i/100)+1)
totallR = round((totallR-1)*100)
if ng > 0:
    avgGain = gains/ng
    maxR = str(max(percentchange))
else:
    avgGain = 0
    maxR = 'undefined'

if nl>0:
    avgLoss = losses/nl
    maxL = str(min(percentchange))
    ratio = str(-(avgGain/avgLoss))
else:
    avgLoss = 0
    maxL = 'undefined'
    ratio = 'inf'

if ng >0 and nl >0:
    bettingAvg = ng/ng+nl
else:
    bettingAvg = 0
    
    
print()
print('Result for'+stock+"going back to "+str(df.index[0])+" Sample size: "+str(ng+nl)+"trades")
print('EMAs used : ',str(emasUsed))
print('Batting Avg : '+str(bettingAvg))
print('Gain/loss ratio: ' + ratio)
print('Avg gain: ' + str(avgGain))
print('Avg loss: '+ str(avgLoss))
print('Max return: '+ str(maxR))
print('Max loss: ' + str(maxL))
print('Total return over '+str(ng+nl)+' trades: '+str(totallR)+'%')
print()


Result forSUNWgoing back to 2020-08-04 11:10:00-04:00 Sample size: 16trades
EMAs used :  [26, 50]
Batting Avg : 13.0
Gain/loss ratio: 21.34279774282376
Avg gain: 99.83509380882168
Avg loss: -4.6776947901495225
Max return: 341.19291580166254
Max loss: -7.760166032369876
Total return over 16 trades: 297.0%



In [148]:
## print results

gains = 0
ng = 0
losses = 0
nl = 0
totallR = 1
for i in percentchange:
    if i >0:
        gains +=i
        ng +=1
    else:
        losses +=i
        nl +=1
    totallR = totallR*((i/100)+1)
totallR = round((totallR-1)*100)
if ng > 0:
    avgGain = gains/ng
    maxR = str(max(percentchange))
else:
    avgGain = 0
    maxR = 'undefined'

if nl>0:
    avgLoss = losses/nl
    maxL = str(min(percentchange))
    ratio = str(-(avgGain/avgLoss))
else:
    avgLoss = 0
    maxL = 'undefined'
    ratio = 'inf'

if ng >0 or nl >0:
    bettingAvg = ng/ng+nl
else:
    bettingAvg = 0
    
    
print()
print('Result for'+stock+"going back to "+str(df.index[0])+" Sample size: "+str(ng+nl)+"trades")
print('EMAs used : ',str(emasUsed))
print('Batting Avg : '+str(bettingAvg))
print('Gain/loss ratio: ' + ratio)
print('Avg gain: ' + str(avgGain))
print('Avg loss: '+ str(avgLoss))
print('Max return: '+ str(maxR))
print('Max loss: ' + str(maxL))
print('Total return over '+str(ng+nl)+' trades: '+str(totallR)+'%')
print()


Result forSAVAgoing back to 2020-08-04 11:10:00-04:00 Sample size: 16trades
EMAs used :  [26, 50]
Batting Avg : 10.0
Gain/loss ratio: 4.631584700957396
Avg gain: 7.312372733581321
Avg loss: -1.5788057880210586
Max return: 29.032253318365232
Max loss: -3.6224680166096146
Total return over 16 trades: 39.0%



In [119]:
gains

160.13534916283754

In [120]:
ng

11

In [121]:
losses

-35.93427437755373

In [122]:
nl

20

In [131]:
ng/(nl)

0.55

In [125]:
ratio

'8.10243661071219'

In [133]:
abs(gains/losses)

4.456340135891704

In [134]:
ratio

'8.10243661071219'

In [ ]:
ng/nl.   gains/lossess

In [142]:
1.5*13

19.5

In [143]:
19.5/13

1.5

In [149]:
1/4

0.25